<a href="https://colab.research.google.com/github/xprilion/flower-species-recognition/blob/master/PlantSpeciesRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 


import tarfile
import urllib
import os
import re

np.random.seed(1)

     |████████████████████████████████| 349.0MB 50kB/s 
     |████████████████████████████████| 3.1MB 32.6MB/s 
     |████████████████████████████████| 430kB 44.1MB/s 
     |████████████████████████████████| 61kB 25.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11


In [2]:
url = "http://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz"
print ("\ndownloading flower images...")    
filename, headers = urllib.request.urlretrieve(
    url,filename=os.path.expanduser("17flowers.tgz"))    
print ("download complete!")

os.chdir(os.path.expanduser("./"))
print ("extracting flower images...")
tar = tarfile.open(os.path.expanduser("17flowers.tgz"), "r:gz")
tar.extractall()
tar.close()
print ("extract complete!")

print ("downloading tensorflow component...")
urllib.request.urlretrieve("https://raw.githubusercontent.com/tensorflow/models/master/tutorials/image/imagenet/classify_image.py",
                       filename=os.path.expanduser("classify_image.py"))
print ("download complete!")

os.chdir(os.path.expanduser("./"))
print ("generating graph...")
os.system("python classify_image.py --model_dir graph/")
print ("graph complete!\n")


downloading flower images...
download complete!
extracting flower images...
extract complete!
downloading tensorflow component...
download complete!
generating graph...
graph complete!



In [14]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True, cache_subdir='/content',)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 1s 0us/step


In [0]:
from __future__ import absolute_import

In [16]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [17]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

In [18]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [22]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
      base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [26]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10
46/46 [==============================] - 409s 9s/step - loss: 0.9702 - accuracy: 0.6853 - val_loss: 0.7938 - val_accuracy: 0.7141
Epoch 2/10
46/46 [==============================] - 400s 9s/step - loss: 0.3872 - accuracy: 0.8588 - val_loss: 0.8534 - val_accuracy: 0.6963
Epoch 3/10
46/46 [==============================] - 394s 9s/step - loss: 0.2930 - accuracy: 0.8969 - val_loss: 0.7214 - val_accuracy: 0.7688
Epoch 4/10
46/46 [==============================] - 399s 9s/step - loss: 0.2149 - accuracy: 0.9204 - val_loss: 0.7707 - val_accuracy: 0.7674
Epoch 5/10
46/46 [==============================] - 398s 9s/step - loss: 0.1921 - accuracy: 0.9309 - val_loss: 0.8810 - val_accuracy: 0.7456
Epoch 6/10
46/46 [==============================] - 397s 9s/step - loss: 0.1588 - accuracy: 0.9428 - val_loss: 0.8424 - val_accuracy: 0.7551
Epoch 7/10
46/46 [==============================] - 400s 9s/step - loss: 0.1192 - accuracy: 0.9609 - val_loss: 0.8635 - val_accuracy: 0.7606
Epoch 8/10
46

In [0]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')